In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from CinePred.new_model import *
from CinePred.data.preprocessing import *
from CinePred.data.importing import import_data
from CinePred.data.featuring import add_inflation

from sklearn.model_selection import cross_val_score, GridSearchCV
from xgboost import XGBRegressor, plot_importance
from matplotlib import pyplot
import numpy as np
from sklearn.inspection import permutation_importance

In [3]:
# install googletrans using pip
#!pip install googletrans
#!pip install translate

In [4]:
import googletrans
from googletrans import Translator

In [5]:
df = import_data('../raw_data/IMDb_movies.csv')
df = preproc(df, path="../raw_data/cat_acteur.csv")

In [31]:
df.shape

(11875, 45)

In [19]:
titles = df[['imdb_title_id', 'title']]

In [20]:
import requests

def fetch_title(imdb_id):
    """
    Get traduction title from The Movie Database API. Returns empty string if song not found
    """
    url = f'https://api.themoviedb.org/3/movie/{imdb_id}/translations?api_key=8fb2a302750a5c0fe27e4ce4d59359e1'
    response = requests.get(url)
    if response.status_code != 200:
        return ''
    data = response.json()
    data = data['translations']
    for i in range(len(data)):
        if data[i]['english_name'] == "English":
            return data[i]['data']['title']
    return ""

In [21]:
# translated = GoogleTranslator(source='auto', target='en').translate("A Memória que me Contam")  # output -> Weiter so, du bist großartig

In [22]:
titles['translation'] = ''
for row in range(len(titles)):
    # Adding all the translations to a dictionary (translations)
    titles.loc[row, 'translation'] = fetch_title(titles.loc[row, 'imdb_title_id'])
    print(titles.loc[row, 'translation'])
titles

/tmp/ipykernel_11866/2554672824.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  titles['translation'] = ''
/home/pierre/.pyenv/versions/3.8.12/envs/cinepred/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Memories They Told Me
Blood Lines
A Paris Education
Kid
Salty Air


Return

How to Get Rid of the Others

Requirements to Be a Normal Person
The Last Lear



Slugs

7lives
I Embrace You With 1000 Arms



Dreams



Lora from Morning Till Evening

Atrocious
Open My Heart




Year of the Nail

Thirty


Selfie
Rough





What Became of Us


To Die in San Hilario
The Circle




The Puffy Chair
The Collectress

Agent Sai Srinivasa Athreya

The Terrorist







Food and Shelter





Frozen Days
Burrowing

10 Days Before the Wedding
Kaili Blues

Oru Adaar Love

Changing Emma
An Episode in the Life of an Iron Picker




Me

Falkenberg Farewell


Pistol, Suitcase and Three Stinking Barrels












Two Lottery Tickets
Carmina or Blow Up
They Will Return

Mila from Mars

Ottam


Sarvann

Noura's Dream
Nikka Zaildar




Dark Sea

Quiet as a Mouse







The Last House in the Woods




Illegitimate

Drained


Descendents

The House

Sirens in the Aegean



In Between Days





Solaris
Aarya


No


Across the Waters
Night and Day
Black Sea

Love and Revolution






Scapegoat




Jasminum
Happy Family


The Merchant of Four Seasons
The Bitter Tears of Petra von Kant
Help, We Are Graduating!
Golden Men
The Dead Queen
Sara's Notebook
Alone at My Wedding



Slovenia, Australia and Tomorrow the World
What the Waters Left Behind
Clandestine Childhood
Hush
Stronger Than The World: The Story of José Aldo




Kya Love Story Hai

Kharms
Women Who Passed My Way

Montparnasse Bienvenüe










Hong Kong Confidential








Ivan Veramathiri



Yellow Flowers On Green Grass
The Here After

Beyond America
The Guard Post
Los bastardos


Spiral



The Summit: A Chronicle Of Stones to Serenity

Zone of the Dead


The Wedding Game



Black Sheep
Dogs
Hedi
A Song of Lisbon


Katia's Sister



One Night in Oslo


Gone
Orange Love
Other Girls
Bunny the Killer Thing
Stranded
Birdboy: The Forgotten Children
Potato Salad: Don't Ask!
One Floor Below

Reckless
Cats Don't Have Vertigo
Hush Hush Baby

I

The Little Traitor

Rarandoi Veduka Chudham
Vikram Vedha
Fidaa
Lapland Odyssey 3
Bad Family
Western

V2: Dead Angel




Slim Susie
Rams
On the Edge


Mountain Patrol
Brasserie Romance
Maria's Men

A Patriotic Man
Grown Ups
The Home of Dark Butterflies
Skirt Day

Graduation

The Animal's Wife

I am not Salvador


8 Best Dates








Antikiller


The Way I Spent the End of the World
In Paris

1895 in Formosa
The Liverpool Goalie

Round Trip

We Are Pregnant
The Legend of Seven Cutter
On the Edge




Disco Polo



What Men Do! 2

Mourning Grave
The Exciled Cow



Corporate Event
The Guide






72 Meters
A Midsummer of Love


Spy Time

Mummy, I'm a Zombie

The Midwife
Toss-Up

The Crossing
37½



The German Doctor



Fratricide

Eleven Men Out

Vinnaithaandi Varuvaayaa


The Vanishing Point
Next Door
August Fools
Here's the Deal
A Pact
The Role

Turn Me On, Dammit!

Jellyfish





Kissed by Winter
Nine Queens





Stone




Celestial Wives of the Meadow Mari





Effi Briest
Temporary Re









My Tutor Friend 2


Christmas Story


A Tram in SP
Road to Istanbul

Under the North Star


Little Longnose
Journey to the Christmas Star



The Night Before the Exams
Pushpaka Vimana
Alleluia
Echoes from the Dead
Rab Ne Bana Di Jodi
The Chessplayer
Little Tito and the Aliens
Operation Golden Shell

Evil Words



Crook




The Miracle of Marcelino

Beyond



Pattathu Yaanai

Bank Robber


All That Matters Is Past


House of Fools

I Murder Seriously
The Last Train
The Tale of Fedot, the Shooter

Sorrow and Joy


Brass Knuckle Boys

Lovey-Dovey 2








Wild Tales


My Grandpa the Bankrobber
Bennie Brat

Look Who's Back
Pokkiri Raja
Pulimurugan
The Assassin Next Door

Like a Dream
The Protector
Sophie's Revenge





The Dolphin: Story of a Dreamer


The Christmas Party




Don't Blame Karma on What Happens to You for Being an Asshole


A Stroke of Luck
Shake It
The Sandman





Stranded

Badlapur
Schneider vs. Bax
Stop Me Here
Happy 140
The Famous Five 4



The Famous Five 3




A Second Chance
Crazy Romance





Cetto C'è

Nikitich and The Dragon

The Magic Crystal
Huevos: Little Rooster's Egg-Cellent Adventure
Remo


Kathmandu Lullaby
The Heineken Kidnapping


The Role of Her Life

Land of Mine




Sleepless
Jaya Janaki Nayaka
Manitou's Shoe
10 Endrathukulla

Summertime
The Counterfeiters

One Hundred Nails
When Pigs Have Wings

See You in Montevideo



The Invisible Guardian
Rescue Under Fire





Yaroslav. A Thousand Years Ago








Camarón: When Flamenco Became Legend


Happy Camper
R... Rajkumar


The Quake


The Accidental Gangster






Dostana





Mohabbatein
I Want You
The Pleasure Is All Mine
Rajapattai
Everybody Has Secrets
Dhoom 2
I Saw the Sun

Yulenka





Mother

High Security Vacation
Castaway on the Moon
Thirst


Gnomes and Trolls: The Secret Chamber
A Dirty Carnival

A Serious Game
Sweet Dreams
Smoke & Mirrors
Good Guys Go to Heaven, Bad Guys Go to Pattaya


Tezz
The Purity of Vengeance

The Crown Jewels





Standing Tall

Night Peacock


Everything is Love















The Legacy
Riding Alone for Thousands of Miles



Home from Home – Chronicle of a Vision
Under the Rainbow


When Did You Last See Your Father?




Paragraph 78: Film One
Flash Point



The Berlin File





Sivaji: The Boss



The Bridgeman


2009: Lost Memories
God Loves Caviar
A Flat for Three

24
Dangal




The Promise

Cyborg She

God's Smile







Aida of the Trees


We Love You, You Bastard






Confession of Pain







Finding Altamira



The Apparition
Sunset
















Hurricane





T-34







The Brats


The Detective 2

The First Beautiful Thing
The Last Days on Mars
The Tunnel
The Admiral: Roaring Currents



The White Masai

Young, Beautiful and Screwed Up

Torrente 5












The Brand New Testament


Armed Hands












Gagarin: First in Space






















Counter Investigation
With Love... from the Age of Reason

Little Spirou
Tad, the Lost Explorer, and the Secret of King Midas
Valley of the Wolves: Iraq

Frie






















The Myth













Assembly

The Heart of the Earth




















The Maidens' Conspiracy
OSS 117: Cairo, Nest of Spies
The Illusionist


City Under Siege







China Strike Force



With Open Arms




The Tiger of Eschnapur
















Battle of the Warriors

Tini: The New Life of Violetta

Silver Hawk










Abigail




























1911

Kalank











Days of Glory








Jet Lag








The Crimson Rivers




Strange Gardens








Throne of Elves








Fellini Satyricon
Turning Tide














A Distant Neighborhood
Let the Bullets Fly






House of Flying Daggers



A Perfect Plan








Secret Agents












A Chinese Odyssey Part Three


Robin-B-Hood

Love Street







Tiger Zinda Hai





War









Porky's 3: Revenge







Fame
The White Ribbon


Broken Embraces









Buddenbrooks




Air Bound


Lady of the Dynasty




Gone with the Bullets













Yamakasi


Metropolis











Anything for Alic








Dragon Blade







































Operation Red Sea



















Asterix & Obelix Take on Caesar

Little Nemo: Adventures in Slumberland









The Monkey King 2

War and Peace











































The Monkey King 3































Asterix & Obelix: God Save Britannia

















A Very Long Engagement
















































Asterix & Obelix: Mission Cleopatra

































The Eight Hundred


















































































Star Trek: Nemesis


Arthur 3: The War of the Two Worlds






















Arthur and the Revenge of Maltazard

















































Two Brothers





















































































Red Cliff Part II














































Bad Company











The Indian Tomb






































R

,imdb_title_id,title,translation
0,tt2476186,A Memória que me Contam,Memories They Told Me
1,tt8128618,Linhas de Sangue,Blood Lines
2,tt6593240,Mes provinciales,A Paris Education
3,tt7991628,Mali,Kid
4,tt0805488,L'aria salata,Salty Air
...,...,...,...
11870,tt4154796,Avengers: Endgame,
11871,tt0056937,Cleopatra,
11872,tt0449088,Pirati dei Caraibi - Ai confini del mondo,
11873,tt0051852,La legge,The Law


In [27]:
titles['both'] = titles['title']
titles.head()

/tmp/ipykernel_11866/3595408641.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  titles['both'] = titles['title']


,imdb_title_id,title,translation,both
0,tt2476186,A Memória que me Contam,Memories They Told Me,A Memória que me Contam
1,tt8128618,Linhas de Sangue,Blood Lines,Linhas de Sangue
2,tt6593240,Mes provinciales,A Paris Education,Mes provinciales
3,tt7991628,Mali,Kid,Mali
4,tt0805488,L'aria salata,Salty Air,L'aria salata


In [28]:
for i in range(len(titles)):
    if titles.loc[i, 'translation'] != '':
        titles.loc[i, 'both'] = titles.loc[i, 'translation']

/home/pierre/.pyenv/versions/3.8.12/envs/cinepred/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [34]:
df = df.merge(right=to_merge, on='imdb_title_id', how='left')
df.head(10)

,worlwide_gross_income,year,duration,budget,production_company,director,writer,shifted,nb_movies_actor1,nb_movies_actor2,...,Horror,Family,Music,Sci-Fi,Animation,Comedy,Nb_actuals_movie_production_company,Nb_actuals_movie_directors,Nb_actuals_movie_writers,both
0,4.476463,2012,95,2.304875,Elixir Entretenimento,Lúcia Murat,"Tatiana Salem Levy, Lúcia Murat",0,1,1.0,...,0,0,0,0,0,0,1,2,1,Memories They Told Me
1,4.908817,2018,132,2.761312,Caos Calmo Filmes,"Sérgio Graciano, Manuel Pureza","Ricardo Oliveira, Manuel Pureza",0,1,1.0,...,0,0,0,0,0,1,1,1,1,Blood Lines
2,5.059217,2018,137,2.840493,Moby Dick Films,Jean-Paul Civeyrac,Jean-Paul Civeyrac,0,1,1.0,...,0,0,0,0,0,0,2,1,1,A Paris Education
3,4.336624,2018,90,2.875255,Propeler Film,Antonio Nuic,Antonio Nuic,0,1,1.0,...,0,0,0,0,0,0,1,1,1,Kid
4,5.825793,2006,87,2.935861,Bianca Film,Alessandro Angelini,"Alessandro Angelini, Angelo Carbone",0,1,1.0,...,0,0,0,0,0,0,2,1,1,Salty Air
5,4.006221,2003,90,3.004289,Fictionworks Production,Roy Finch,Roy Finch,0,1,1.0,...,0,0,0,0,0,0,1,1,1,Wake
6,3.793392,2011,94,3.082107,Source Productions,Terry McMahon,Terry McMahon,0,1,1.0,...,0,0,0,0,0,0,1,1,1,Charlie Casanova
7,6.756191,2007,113,3.096295,Beautiful Pictures,Kyu-maan Lee,"Hyeon-jin Lee, Kyu-maan Lee",0,1,1.0,...,0,0,0,0,0,0,1,1,1,Return
8,4.462422,2011,76,3.102311,Driving With Our Eyes Shut,Glenn Berggoetz,Glenn Berggoetz,0,1,1.0,...,0,0,0,0,0,1,1,1,1,The Worst Movie Ever!
9,5.115208,2007,94,3.137688,Zentropa Entertainments,Anders Rønnow Klarlund,"Rasmus Botoft, Anders Rønnow Klarlund",0,1,1.0,...,0,0,0,1,0,1,5,1,1,How to Get Rid of the Others


In [41]:
df.to_csv('preprocessed.csv', index=False)

In [39]:
df.drop(columns='title', inplace=True)
df.columns

Index(['worlwide_gross_income', 'year', 'duration', 'budget',
       'production_company', 'director', 'writer', 'shifted',
       'nb_movies_actor1', 'nb_movies_actor2', 'nb_movies_actor3',
       'last income', 'imdb_title_id', 'actors', 'description', 'avg_vote',
       'country', 'date_sin', 'date_cos', 'Action', 'Biography', 'Thriller',
       'Crime', 'Drama', 'Adventure', 'Romance', 'Western', 'Documentary',
       'Musical', 'Film-Noir', 'Fantasy', 'Mystery', 'Sport', 'History', 'War',
       'Horror', 'Family', 'Music', 'Sci-Fi', 'Animation', 'Comedy',
       'Nb_actuals_movie_production_company', 'Nb_actuals_movie_directors',
       'Nb_actuals_movie_writers', 'both'],
      dtype='object')

In [40]:
df.rename(columns={'both': 'title'}, inplace=True)
df

,worlwide_gross_income,year,duration,budget,production_company,director,writer,shifted,nb_movies_actor1,nb_movies_actor2,...,Horror,Family,Music,Sci-Fi,Animation,Comedy,Nb_actuals_movie_production_company,Nb_actuals_movie_directors,Nb_actuals_movie_writers,title
0,4.476463,2012,95,2.304875,Elixir Entretenimento,Lúcia Murat,"Tatiana Salem Levy, Lúcia Murat",0,1,1.0,...,0,0,0,0,0,0,1,2,1,Memories They Told Me
1,4.908817,2018,132,2.761312,Caos Calmo Filmes,"Sérgio Graciano, Manuel Pureza","Ricardo Oliveira, Manuel Pureza",0,1,1.0,...,0,0,0,0,0,1,1,1,1,Blood Lines
2,5.059217,2018,137,2.840493,Moby Dick Films,Jean-Paul Civeyrac,Jean-Paul Civeyrac,0,1,1.0,...,0,0,0,0,0,0,2,1,1,A Paris Education
3,4.336624,2018,90,2.875255,Propeler Film,Antonio Nuic,Antonio Nuic,0,1,1.0,...,0,0,0,0,0,0,1,1,1,Kid
4,5.825793,2006,87,2.935861,Bianca Film,Alessandro Angelini,"Alessandro Angelini, Angelo Carbone",0,1,1.0,...,0,0,0,0,0,0,2,1,1,Salty Air
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11870,9.459941,2019,181,8.564574,Marvel Studios,"Anthony Russo, Joe Russo","Christopher Markus, Stephen McFeely",16,34,16.0,...,0,0,0,0,0,0,12,6,8,Avengers: Endgame
11871,8.689022,1963,192,8.570714,Twentieth Century Fox,Joseph L. Mankiewicz,"Joseph L. Mankiewicz, Ranald MacDougall",1,1,32.0,...,0,0,0,0,0,0,18,5,1,Cleopatra
11872,9.079017,2007,169,8.573417,Walt Disney Pictures,Gore Verbinski,"Ted Elliott, Terry Rossio",6,9,4.0,...,0,0,0,0,0,0,63,7,3,Pirati dei Caraibi - Ai confini del mondo
11873,5.188414,1959,126,8.834959,Le Groupe des Quatre,Jules Dassin,"Roger Vailland, Jules Dassin",0,1,1.0,...,0,0,0,0,0,1,1,1,1,The Law


In [35]:
df.shape

(11875, 46)

In [33]:
to_merge = titles[['imdb_title_id', 'both']]
to_merge

,imdb_title_id,both
0,tt2476186,Memories They Told Me
1,tt8128618,Blood Lines
2,tt6593240,A Paris Education
3,tt7991628,Kid
4,tt0805488,Salty Air
...,...,...
11870,tt4154796,Avengers: Endgame
11871,tt0056937,Cleopatra
11872,tt0449088,Pirati dei Caraibi - Ai confini del mondo
11873,tt0051852,The Law


In [30]:
titles.tail()

,imdb_title_id,title,translation,both
11870,tt4154796,Avengers: Endgame,,Avengers: Endgame
11871,tt0056937,Cleopatra,,Cleopatra
11872,tt0449088,Pirati dei Caraibi - Ai confini del mondo,,Pirati dei Caraibi - Ai confini del mondo
11873,tt0051852,La legge,The Law,The Law
11874,tt0048308,Lola Montès,,Lola Montès


In [ ]:
# Replacing all the translated words from the dictionary to the original dataframe
titles.replace(translations, inplace = True)
titles.head(10)

##### Chapeau

In [ ]:
import pandas as pd

In [ ]:
acteurs_df = pd.read_csv("../raw_data/cat_acteur.csv")

In [ ]:
acteurs_df.head()

In [ ]:
acteurs_df.ratio.plot.()

In [ ]:
import matplotlib.pyplot as plt
acteurs_df.groupby(by='title').sum()['ratio'].plot.hist(bins=100)


In [ ]:
# acteurs_df.groupby(by='acteur_name').mean()

# acteurs_df.groupby(by='acteur_name').max()['shifted'].sort_values(ascending=False).head(10)

# acteurs_df.groupby(by='acteur_name').mean()['income'].sort_values(ascending=False).head(10)

# acteurs_df.groupby(by='acteur_name').mean()['budget'].sort_values(ascending=False).head(10)

# acteurs_df.groupby(by='acteur_name').mean()['ratio'].sort_values(ascending=False).head(10)

In [ ]:
acteurs_df['year'] = convert_to_int(acteurs_df[['year']])
acteurs_df.sort_values(by='year', inplace=True)
acteurs_df['ones'] = 1
acteurs_df['last income'] = acteurs_df['income']

In [ ]:
acteurs_df['last income'] = acteurs_df.groupby(by='acteur_name')['last income'].transform(lambda x: x.shift(1, fill_value=0))
acteurs_df

In [ ]:
#actors.rename(columns={"income" : "last_income"}, inplace=True)

In [ ]:
#acteurs_df[acteurs_df.acteur_name =='Leonardo DiCaprio']

In [ ]:
acteurs_df = acteurs_df.groupby(by='title').agg({'last income' : 'sum'})
acteurs_df

In [ ]:
acteurs_df.reset_index(inplace=True)
acteurs_df = acteurs_df.rename(columns = {'index':'acteur_name'})
acteurs_df

In [ ]:
df = df.merge(right=acteurs_df, on='title', how="left")
df

In [ ]:
acteurs_df[acteurs_df.acteur_name =='Leonardo DiCaprio']

#### Clean

In [ ]:
# NA & columns:
df = add_success_movies_per_actors(df, path = "../raw_data/cat_acteur.csv")
    
df = keep_columns(df,
                      column_names=[
                          'year', 'date_published', 'genre', 'duration',
                          'budget', 'worlwide_gross_income',
                          'production_company', 'director', 'writer', 'shifted'
                      ])
df = remove_na_rows(df)

In [ ]:
# date_published
df['date_published'] = convert_to_date(df[['date_published']])

# day of the year
df['date_sin'] = add_sin_features(df[['date_published']])
df['date_cos'] = add_cos_features(df[['date_published']])
df.drop(columns='date_published', inplace=True)

# year, duration
df['year'] = convert_to_int(df[['year']])
df['duration'] = convert_to_int(df[['duration']])

# genre
ohe = GenreOHE()
ohe.fit(df) # la colonne 'genre' est spécifié dans la classe
df = ohe.transform(df)

# budget
df['budget'] = convert_budget_column(df[['budget']])
df = df[df['budget'] > 100]
df = add_inflation(df, 'budget')
df['budget'] = log_transformation(df[['budget']])

# income
df['worlwide_gross_income'] = convert_income(df[['worlwide_gross_income']])
df = add_inflation(df, 'worlwide_gross_income')
df['worlwide_gross_income'] = log_transformation(df[['worlwide_gross_income']])

# cumsum
df = Add_Ones(df)
df = Add_number_of_movies_per_prod_company_in_Timeline(df)
df = Add_number_of_movies_per_directors_in_Timeline(df)
df = Add_number_of_movies_per_writer_in_Timeline(df)
df = Remove_Ones(df)

# sort & index:
df.sort_values('budget', inplace=True)
df.reset_index(inplace=True)
df.drop(columns=['index', 'production_company', 'director', 'writer'],
        inplace=True)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#### Model fit

In [ ]:
mid = int(df.shape[0] / 2)
df1 = df.iloc[:mid].copy()
df2 = df.iloc[mid:].copy()

In [ ]:
X = df1.drop(columns=['worlwide_gross_income', 'actors', 'production_company', 'director', 'writer', 'imdb_title_id', 'description', 'title', 'avg_vote', 'country'])
y = df1['worlwide_gross_income']

model = XGBRegressor(learning_rate=0.1, max_depth=2)

cv = cross_val_score(model, X, y, cv=5, scoring='neg_mean_absolute_error')

In [ ]:
X2 = df2.drop(columns=['worlwide_gross_income', 'actors', 'production_company', 'director', 'writer', 'imdb_title_id', 'description', 'title', 'avg_vote', 'country'])
y2 = df2['worlwide_gross_income']
model2 = XGBRegressor(learning_rate=0.1, max_depth=2, n_estimators=300)
cv2 = cross_val_score(model2, X2, y2, cv=5, scoring='neg_mean_absolute_error')

In [ ]:
#score1 = np.mean(np.abs(cv)) # {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 200}

In [ ]:
score2 = np.mean(np.abs(cv2))

In [ ]:
print(score1, score2)

In [ ]:
X2.shape

#### Feature importance

In [ ]:
model2.fit(X2, y2)

In [ ]:
permutation_score = permutation_importance(model2, X2, y2, n_repeats=10) # Perform Permutation

importance_df = pd.DataFrame(np.vstack((X2.columns,
                                        permutation_score.importances_mean)).T) # Unstack results
importance_df.columns=['feature','score decrease']

importance_df.sort_values(by="score decrease", ascending = False) # Order by importance

In [ ]:
plot_importance(model2, importance_type='gain')
pyplot.show()

In [ ]:
d = {'name': list(X.columns), 'importance': list(model.feature_importances_)}

data = pd.DataFrame(d)

data.sort_values(by="importance", ascending=False)

In [ ]:
X = X.drop(columns=['Music', "Thriller", 'date_sin', 'Action', 'Documentary', 'Film-Noir', 'Fantasy', 'Musical', 'Western', 'Crime'])